In [19]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [20]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [21]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

9.999999999998899e-05

In [22]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [23]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df['pivots_l'] = False
    df['pivots_h'] = False

    df['low_perv'] = df.low.shift(1)
    df['high_perv'] = df.high.shift(1)
    df['low_next'] = df.low.shift(-1)
    df['high_next'] = df.high.shift(-1)
    df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
    df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df.pivots_l = df.pivots_l.astype(float)
    df.pivots_h = df.pivots_h.astype(float)
    return df

In [24]:
df_an = featureExtraction(df_an)

In [25]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    break
        
        df_an['SL'][i] = df_an.low[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
                    
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    break
                    
        df_an['SL'][i] = df_an.high[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    

C:\Users\meysam\AppData\Local\Temp\ipykernel_10432\3582309630.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.low[i-1]
C:\Users\meysam\AppData\Local\Temp\ipykernel_10432\3582309630.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
C:\Users\meysam\AppData\Local\Temp\ipykernel_10432\3582309630.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [26]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'full_range', 'body_lower',
       'body_upper', 'body_bottom_perc', 'body_top_perc', 'body_perc',
       'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'BB_MA_cross', 'BB_UP_touch',
       'BB_LW_touch', 'EMA_cross', 'KeUp_touch', 'KeLo_touch', 'RSI_level',
       'MACDHIST_level', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [27]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level','MACDHIST_level',
    
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [28]:
X = []
Y = [] 
indeces = []
for i in range(4,len(df_an)):
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])
#             feature.append(df_an[fc][i-5])
#             feature.append(df_an[fc][i-6])
#             feature.append(df_an[fc][i-8])
#             feature.append(df_an[fc][i-9])
#             feature.append(df_an[fc][i-10])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [ ]:
feature_name = []
feature_name.append('pivots_h[i-1]')
feature_name.append('pivots_l[i-1]')
feature_name.append('pivots_h[i-2]')
feature_name.append('pivots_l[i-2]')
feature_name.append('pivots_h[i-3]')
feature_name.append('pivots_l[i-3]')
feature_name.append('pivots_h[i-4]')
feature_name.append('pivots_l[i-4]')
for fc in feature_col:
    feature_name.append(fc+'[i]')
    feature_name.append(fc+'[i-1]')
    feature_name.append(fc+'[i-2]')
    feature_name.append(fc+'[i-3]')
    feature_name.append(fc+'[i-4]')

In [ ]:
# for i in range(len(feature_name)):
#     print(feature_name[i]," : ",feature[i])

In [29]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]

In [31]:
len(feature)

98

In [34]:
clf = MLPClassifier( solver='adam', alpha=1e-4,learning_rate_init=1e-4, # adam
                    hidden_layer_sizes=(100,100), 
                    random_state=1,
                    max_iter=2000,
                    verbose=True,warm_start = True,
                    n_iter_no_change=10,
#                     early_stopping = True,
#                     validation_fraction = 0.2 # default is 0.1
                   )

In [35]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 1.03206502
Iteration 2, loss = 0.94530999
Iteration 3, loss = 0.91198732
Iteration 4, loss = 0.88846470
Iteration 5, loss = 0.86777851
Iteration 6, loss = 0.84931590
Iteration 7, loss = 0.83277397
Iteration 8, loss = 0.81807426
Iteration 9, loss = 0.80413798
Iteration 10, loss = 0.79182790
Iteration 11, loss = 0.78043323
Iteration 12, loss = 0.77043479
Iteration 13, loss = 0.76112846
Iteration 14, loss = 0.75257337
Iteration 15, loss = 0.74483357
Iteration 16, loss = 0.73779774
Iteration 17, loss = 0.73155312
Iteration 18, loss = 0.72597294
Iteration 19, loss = 0.72104673
Iteration 20, loss = 0.71658508
Iteration 21, loss = 0.71257506
Iteration 22, loss = 0.70892951
Iteration 23, loss = 0.70554496
Iteration 24, loss = 0.70300200
Iteration 25, loss = 0.69948333
Iteration 26, loss = 0.69690633
Iteration 27, loss = 0.69432947
Iteration 28, loss = 0.69219270
Iteration 29, loss = 0.69002679
Iteration 30, loss = 0.68805011
Iteration 31, loss = 0.68602036
Iteration 32, los

Iteration 253, loss = 0.45047317
Iteration 254, loss = 0.44920652
Iteration 255, loss = 0.44786776
Iteration 256, loss = 0.44665928
Iteration 257, loss = 0.44689251
Iteration 258, loss = 0.44428644
Iteration 259, loss = 0.44401486
Iteration 260, loss = 0.44302380
Iteration 261, loss = 0.44330784
Iteration 262, loss = 0.44122714
Iteration 263, loss = 0.44082393
Iteration 264, loss = 0.44054120
Iteration 265, loss = 0.43910779
Iteration 266, loss = 0.43799991
Iteration 267, loss = 0.43713656
Iteration 268, loss = 0.43600803
Iteration 269, loss = 0.43556335
Iteration 270, loss = 0.43417935
Iteration 271, loss = 0.43457505
Iteration 272, loss = 0.43345888
Iteration 273, loss = 0.43208316
Iteration 274, loss = 0.43144617
Iteration 275, loss = 0.43014438
Iteration 276, loss = 0.43029942
Iteration 277, loss = 0.42862787
Iteration 278, loss = 0.42734083
Iteration 279, loss = 0.42631821
Iteration 280, loss = 0.42668463
Iteration 281, loss = 0.42533788
Iteration 282, loss = 0.42461039
Iteration 

Iteration 502, loss = 0.26814890
Iteration 503, loss = 0.26767971
Iteration 504, loss = 0.26712342
Iteration 505, loss = 0.26584425
Iteration 506, loss = 0.26586436
Iteration 507, loss = 0.26598235
Iteration 508, loss = 0.26475952
Iteration 509, loss = 0.26374760
Iteration 510, loss = 0.26341400
Iteration 511, loss = 0.26337662
Iteration 512, loss = 0.26242095
Iteration 513, loss = 0.26151191
Iteration 514, loss = 0.26083503
Iteration 515, loss = 0.26099741
Iteration 516, loss = 0.26057461
Iteration 517, loss = 0.25908731
Iteration 518, loss = 0.25930612
Iteration 519, loss = 0.25871170
Iteration 520, loss = 0.25720742
Iteration 521, loss = 0.25689977
Iteration 522, loss = 0.25694442
Iteration 523, loss = 0.25664064
Iteration 524, loss = 0.25583031
Iteration 525, loss = 0.25429767
Iteration 526, loss = 0.25343620
Iteration 527, loss = 0.25347558
Iteration 528, loss = 0.25325241
Iteration 529, loss = 0.25254168
Iteration 530, loss = 0.25300413
Iteration 531, loss = 0.25069086
Iteration 

Iteration 751, loss = 0.14628476
Iteration 752, loss = 0.14518036
Iteration 753, loss = 0.14494184
Iteration 754, loss = 0.14411144
Iteration 755, loss = 0.14408102
Iteration 756, loss = 0.14339450
Iteration 757, loss = 0.14332600
Iteration 758, loss = 0.14248634
Iteration 759, loss = 0.14279329
Iteration 760, loss = 0.14246363
Iteration 761, loss = 0.14087902
Iteration 762, loss = 0.14167444
Iteration 763, loss = 0.14037417
Iteration 764, loss = 0.14064059
Iteration 765, loss = 0.14061679
Iteration 766, loss = 0.13962211
Iteration 767, loss = 0.13861646
Iteration 768, loss = 0.13897889
Iteration 769, loss = 0.13863034
Iteration 770, loss = 0.13823520
Iteration 771, loss = 0.13933759
Iteration 772, loss = 0.13695738
Iteration 773, loss = 0.13673397
Iteration 774, loss = 0.13686145
Iteration 775, loss = 0.13581053
Iteration 776, loss = 0.13578698
Iteration 777, loss = 0.13506700
Iteration 778, loss = 0.13637090
Iteration 779, loss = 0.13406949
Iteration 780, loss = 0.13435160
Iteration 

Iteration 1001, loss = 0.06760676
Iteration 1002, loss = 0.06818088
Iteration 1003, loss = 0.06741574
Iteration 1004, loss = 0.06742425
Iteration 1005, loss = 0.06748926
Iteration 1006, loss = 0.06694166
Iteration 1007, loss = 0.06689028
Iteration 1008, loss = 0.06679544
Iteration 1009, loss = 0.06644332
Iteration 1010, loss = 0.06577494
Iteration 1011, loss = 0.06570312
Iteration 1012, loss = 0.06558087
Iteration 1013, loss = 0.06595215
Iteration 1014, loss = 0.06478401
Iteration 1015, loss = 0.06484402
Iteration 1016, loss = 0.06477518
Iteration 1017, loss = 0.06400272
Iteration 1018, loss = 0.06372172
Iteration 1019, loss = 0.06435011
Iteration 1020, loss = 0.06401274
Iteration 1021, loss = 0.06420116
Iteration 1022, loss = 0.06349722
Iteration 1023, loss = 0.06285449
Iteration 1024, loss = 0.06293125
Iteration 1025, loss = 0.06241441
Iteration 1026, loss = 0.06233242
Iteration 1027, loss = 0.06246094
Iteration 1028, loss = 0.06236764
Iteration 1029, loss = 0.06171585
Iteration 1030

Iteration 1242, loss = 0.02833774
Iteration 1243, loss = 0.02823297
Iteration 1244, loss = 0.02817041
Iteration 1245, loss = 0.02831215
Iteration 1246, loss = 0.02796680
Iteration 1247, loss = 0.02851886
Iteration 1248, loss = 0.02804638
Iteration 1249, loss = 0.02768866
Iteration 1250, loss = 0.02752104
Iteration 1251, loss = 0.02732777
Iteration 1252, loss = 0.02726577
Iteration 1253, loss = 0.02750271
Iteration 1254, loss = 0.02689101
Iteration 1255, loss = 0.02698128
Iteration 1256, loss = 0.02696175
Iteration 1257, loss = 0.02643550
Iteration 1258, loss = 0.02707230
Iteration 1259, loss = 0.02644007
Iteration 1260, loss = 0.02635311
Iteration 1261, loss = 0.02611140
Iteration 1262, loss = 0.02618330
Iteration 1263, loss = 0.02593601
Iteration 1264, loss = 0.02635995
Iteration 1265, loss = 0.02594310
Iteration 1266, loss = 0.02558373
Iteration 1267, loss = 0.02558763
Iteration 1268, loss = 0.02543827
Iteration 1269, loss = 0.02553207
Iteration 1270, loss = 0.02570408
Iteration 1271

C:\Users\meysam\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(100, 100), learning_rate_init=0.0001,
              max_iter=2000, random_state=1, verbose=True, warm_start=True)

In [36]:
y_perdict = clf.predict(X_test)

In [37]:
clf.score(X_train, Y_train)

0.9991341991341991

In [38]:
clf.score(X_test, Y_test)

0.5481823427582228

In [39]:
conf = confusion_matrix(Y_test, y_perdict)
conf

array([[138, 205,   9],
       [197, 651, 195],
       [  9, 168, 161]], dtype=int64)

In [43]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)

C:\Users\meysam\AppData\Local\Temp\ipykernel_10432\3314508941.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL.loc[idx] = y_perdict[i]
C:\Users\meysam\AppData\Local\Temp\ipykernel_10432\3314508941.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL.loc[idx] = y_perdict[i]


In [44]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [45]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.38724035608308605
-1.0    413
 2.0    261
Name: result, dtype: int64


In [57]:
df_res_m5[df_res_m5.TP == 0]

,running,start_index_m5,profit_factor,loss_factor,SPREAD,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,balance,time


In [52]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = 1 # abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

1.8909090909093735
2.3113636363636605
1.9333333333333191
2.0493827160494336
2.0469483568076114
0.11940298507463182
2.397959183673592
0.42105263157790174
2.257352941176437
2.7252747252748675
1.8181818181819016
0.0727272727273388
0.13333333333327166
2.129032258064345
2.044943820224512
1.890624999999317
1.9461538461536674
2.2459016393444173
8.82352941176486
3.5608695652174047
1.9375000000009541
1.9040590405906705
2.1605504587155226
1.9857142857145078
1.9865470852017557
1.6710526315793663
1.7750000000003516
2.192307692307813
2.0100222717149383
2.1081081081078
2.2357954545455505
2.787037037036502
1.9971671388102428
1.9324758842444292
2.1088435374150016
1.9531772575252198
1.9734693877551543
2.897959183673438
2.0198019801977063
2.3963414634148266
2.468085106382657
2.064356435643432
2.2975460122701334
1.9904761904762338
2.1560975609757747
1.9744525547443599
2.055555555555921
2.038277511961707
0.008130081300805914
3.221674876847433
2.162561576354726
2.4763458401304956
4.16074766355146
2.7003012

C:\Users\meysam\AppData\Local\Temp\ipykernel_10432\3725986115.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [51]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])

min balance  9.146859309245672
